In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


In [7]:
disc = pd.read_csv('../dados/disciplinas.csv')
consulta = pd.read_csv('../dados/preferencias.csv')
oferta = pd.read_csv('../dados/solucao_25_1.csv')

In [9]:
#Analisando as preferencias por disciplina - Verifica quantos professores 
a1 = consulta.groupby('id_disciplina')['preferencia'].value_counts().unstack().fillna(0).astype(int).sort_index()
a1 = disc[['id_disciplina', 'disciplina']].merge(a1, left_on='id_disciplina', right_index=True, how='right')#.set_index('disciplina').sort_values(by='codigo_disciplina')
#a11 = a1.reset_index(drop=True).melt(id_vars='codigo_disciplina', var_name='preferencia', value_name='value')

In [13]:
a1 = consulta.merge(disc[['id_disciplina', 'disciplina']], on = 'id_disciplina',how ='left')

In [40]:
a2 = a1.groupby(['disciplina','preferencia']).agg({'id_disciplina':'nunique', 'preferencia':'count'}).rename(columns={'id_disciplina':'n_turmas', 'preferencia':'num_escolhas'}).reset_index().pivot_table(index=['disciplina', 'n_turmas'], columns='preferencia', values='num_escolhas', fill_value=0).reset_index()

a2[0] = (a2[0]/a2['n_turmas']).astype(int)
a2[1] = (a2[1]/a2['n_turmas']).astype(int)
a2[2] = (a2[2]/a2['n_turmas']).astype(int)
a2[3] = (a2[3]/a2['n_turmas']).astype(int)

a2.columns = ['Disciplina', 'Nº Turmas', 'Pref 0', 'Pref 1', 'Pref 2', 'Pref 3']
a2['Disciplina'] = a2['Disciplina'].str.capitalize()

In [53]:
latex_abnt = a2.reset_index().to_latex(
    index=False,
    caption="Distribuição das preferências por disciplina.",
    label="tab:preferencias_disciplina",
    bold_rows=True,
    longtable=True,
    escape=True  # Protege contra caracteres especiais
)
with open("../tabelas/tabela_descritiva_preferencias_disciplina.tex", "w", encoding="utf-8") as f:
    f.write(latex_abnt)

In [54]:
a2.to_clipboard(index=False)

In [20]:
a2 = consulta.groupby('ID')['preferencia'].value_counts().unstack().fillna(0).astype(int).sort_index()
a2 = a2.reset_index(drop=True).reset_index()
a2 = a2.rename(columns={'index':'Docente'})

In [51]:
a3 = consulta.groupby(['id_docente','preferencia']).agg({'preferencia':'count'}).unstack().fillna(0).astype(int).reset_index()
a3.columns = ['Docente', 'Pref 0', 'Pref 1', 'Pref 2', 'Pref 3']

In [56]:
a3.to_clipboard(index=False)

In [52]:
latex_abnt = a3.to_latex(
    index=False,
    caption="Distribuição das preferências por docente.",
    label="tab:preferencias_professores",
    bold_rows=True,
    longtable=True,
    escape=True  # Protege contra caracteres especiais
)
with open("../tabelas/tabela_descritiva_preferencias_professores.tex", "w", encoding="utf-8") as f:
    f.write(latex_abnt)

In [27]:
## Analisando as preferências dos professores sobre as disciplinas
a3 = a2.drop(columns=['Docente']).melt()

fig = px.box(
    a3,
    x='preferencia',
    y='value',
    color='preferencia',
    color_discrete_sequence=px.colors.qualitative.Set2,
    points="all",
    template="seaborn"
)

# Adiciona marcador de média para cada grupo
medias = a3.groupby('preferencia')['value'].mean().reset_index()
fig.add_trace(
    go.Scatter(
        x=medias['preferencia'],
        y=medias['value'],
        mode='markers',
        marker=dict(symbol='diamond', size=8, color='black'),
        name='Média',
        showlegend=True
    )
)

fig.update_layout(
    xaxis_title='Preferência',
    yaxis_title='Nº de Escolhas',
    font=dict(family="Arial", size=16),
    legend_title_text='Preferência',
    showlegend=False,
    autosize=False,
    width=600,
    height=400,
    margin=dict(l=60, r=40, t=60, b=60)
)
fig.show()

In [1]:
import pandas as pd

In [ ]:
sample()